# 04 - Attack Scenarios and Security

This notebook demonstrates how to test federated learning systems against various attacks to evaluate robustness and security.

## What You'll Learn
- Available attack types and their effects
- How to configure and run security experiments
- Evaluating model robustness against attacks
- Best practices for security testing

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

from experiment_runners.run_with_attacks import main as run_attacks
from attacks import (
    noise_injection, label_flipping, gradient_flipping,
    client_failure, data_asymmetry, missed_class
)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## 2. Available Attack Types

The framework supports several attack scenarios to test model robustness:

In [ ]:
# Available attacks and their descriptions
attacks_info = {
    'noise_injection': 'Adds random noise to model parameters',
    'label_flipping': 'Flips labels in training data (data poisoning)',
    'gradient_flipping': 'Flips gradients before aggregation',
    'client_failure': 'Simulates client dropouts and failures',
    'data_asymmetry': 'Creates non-IID data distribution',
    'missed_class': 'Clients missing specific classes'
}

print("Available Attack Scenarios:")
print("=" * 40)
for attack, description in attacks_info.items():
    print(f"• {attack}: {description}")

## 3. Basic Attack Configuration

Configure attacks with different intensity levels:

In [ ]:
# Example attack configurations
attack_configs = {
    'noise_injection': {
        'type': 'noise_injection',
        'intensity': 0.1,  # Noise level (0.0 to 1.0)
        'affected_clients': 0.3  # 30% of clients affected
    },
    'label_flipping': {
        'type': 'label_flipping',
        'flip_rate': 0.2,  # 20% of labels flipped
        'affected_clients': 0.2
    },
    'client_failure': {
        'type': 'client_failure',
        'failure_rate': 0.15,  # 15% client dropout rate
        'random_failures': True
    }
}

print("Attack Configuration Examples:")
for name, config in attack_configs.items():
    print(f"\n{name.upper()}:")
    for key, value in config.items():
        print(f"  {key}: {value}")

## 4. Running Security Experiments

Test model robustness against different attacks:

In [ ]:
# Simple security test function
def run_security_test(attack_type, dataset='MNIST', strategy='FedAvg'):
    """
    Run a security test with specified attack.
    
    Args:
        attack_type: Type of attack to simulate
        dataset: Dataset to use (MNIST, FashionMNIST, CIFAR10)
        strategy: Aggregation strategy
    """
    print(f"Running security test: {attack_type} on {dataset} with {strategy}")
    
    # This would typically call the actual experiment runner
    # For demonstration, we'll simulate results
    baseline_accuracy = np.random.uniform(0.85, 0.95)
    attacked_accuracy = baseline_accuracy - np.random.uniform(0.05, 0.25)
    
    return {
        'baseline_accuracy': baseline_accuracy,
        'attacked_accuracy': max(0, attacked_accuracy),
        'robustness_score': attacked_accuracy / baseline_accuracy if baseline_accuracy > 0 else 0
    }

# Test different attacks
results = {}
test_attacks = ['noise_injection', 'label_flipping', 'client_failure']

for attack in test_attacks:
    results[attack] = run_security_test(attack)
    print(f"\n{attack.upper()} Results:")
    print(f"  Baseline Accuracy: {results[attack]['baseline_accuracy']:.3f}")
    print(f"  Under Attack: {results[attack]['attacked_accuracy']:.3f}")
    print(f"  Robustness Score: {results[attack]['robustness_score']:.3f}")

## 5. Visualizing Attack Impact

In [ ]:
# Create visualization of attack impact
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy comparison
attacks = list(results.keys())
baseline_accs = [results[a]['baseline_accuracy'] for a in attacks]
attacked_accs = [results[a]['attacked_accuracy'] for a in attacks]

x = np.arange(len(attacks))
width = 0.35

ax1.bar(x - width/2, baseline_accs, width, label='Baseline', alpha=0.8)
ax1.bar(x + width/2, attacked_accs, width, label='Under Attack', alpha=0.8)
ax1.set_xlabel('Attack Type')
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Performance: Baseline vs Under Attack')
ax1.set_xticks(x)
ax1.set_xticklabels([a.replace('_', ' ').title() for a in attacks], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Robustness scores
robustness_scores = [results[a]['robustness_score'] for a in attacks]
colors = ['green' if s > 0.8 else 'orange' if s > 0.6 else 'red' for s in robustness_scores]

ax2.bar(attacks, robustness_scores, color=colors, alpha=0.7)
ax2.set_xlabel('Attack Type')
ax2.set_ylabel('Robustness Score')
ax2.set_title('Model Robustness Against Attacks')
ax2.set_xticklabels([a.replace('_', ' ').title() for a in attacks], rotation=45)
ax2.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Good (>0.8)')
ax2.axhline(y=0.6, color='orange', linestyle='--', alpha=0.5, label='Fair (>0.6)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Strategy Robustness Comparison

Compare how different aggregation strategies handle attacks:

In [ ]:
# Test different strategies against attacks
strategies = ['FedAvg', 'FedProx', 'Krum', 'TrimmedMean']
attack_type = 'noise_injection'

strategy_results = {}
for strategy in strategies:
    strategy_results[strategy] = run_security_test(attack_type, strategy=strategy)

# Create robustness comparison
df_results = pd.DataFrame({
    'Strategy': strategies,
    'Baseline': [strategy_results[s]['baseline_accuracy'] for s in strategies],
    'Under Attack': [strategy_results[s]['attacked_accuracy'] for s in strategies],
    'Robustness': [strategy_results[s]['robustness_score'] for s in strategies]
})

print(f"Strategy Robustness Against {attack_type.upper()}:")
print("=" * 50)
print(df_results.round(3).to_string(index=False))

# Find most robust strategy
most_robust = df_results.loc[df_results['Robustness'].idxmax(), 'Strategy']
print(f"\nMost robust strategy: {most_robust}")

## 7. Advanced Attack Scenarios

Test combined attacks and adaptive scenarios:

In [ ]:
# Simulate combined attack scenario
def combined_attack_test():
    """
    Test model against multiple simultaneous attacks.
    """
    print("Testing Combined Attack Scenario:")
    print("- 20% clients with label flipping")
    print("- 15% clients with noise injection")
    print("- 10% client failures")
    
    # Simulate combined attack impact
    baseline = 0.90
    # Combined attacks typically have compounding effects
    combined_impact = 0.15 + 0.10 + 0.05  # Individual impacts don't simply add
    attacked_accuracy = max(0.1, baseline - combined_impact)
    
    return {
        'baseline': baseline,
        'attacked': attacked_accuracy,
        'total_impact': baseline - attacked_accuracy
    }

combined_result = combined_attack_test()
print(f"\nResults:")
print(f"Baseline Accuracy: {combined_result['baseline']:.3f}")
print(f"Under Combined Attack: {combined_result['attacked']:.3f}")
print(f"Total Impact: {combined_result['total_impact']:.3f}")
print(f"Robustness: {combined_result['attacked']/combined_result['baseline']:.3f}")

## 8. Security Best Practices

Key recommendations for secure federated learning:

In [ ]:
# Security recommendations based on attack testing
security_practices = {
    'Strategy Selection': [
        'Use robust aggregation (Krum, TrimmedMean) for Byzantine tolerance',
        'Consider FedProx for heterogeneous environments',
        'Avoid simple FedAvg in adversarial settings'
    ],
    'Client Management': [
        'Implement client authentication and validation',
        'Monitor client behavior for anomalies',
        'Use client sampling to reduce attack surface'
    ],
    'Data Protection': [
        'Validate data quality before training',
        'Use differential privacy when possible',
        'Implement secure aggregation protocols'
    ],
    'Monitoring': [
        'Track model performance degradation',
        'Monitor convergence patterns',
        'Set up alerts for suspicious behavior'
    ]
}

print("FEDERATED LEARNING SECURITY BEST PRACTICES")
print("=" * 50)

for category, practices in security_practices.items():
    print(f"\n{category.upper()}:")
    for practice in practices:
        print(f"  • {practice}")

## 9. Running Real Security Experiments

Use the framework's experiment runner for comprehensive testing:

In [ ]:
# Example of how to run real security experiments
# (Uncomment and modify paths as needed)

example_config = {
    'dataset': 'MNIST',
    'num_clients': 10,
    'num_rounds': 20,
    'strategy': 'FedAvg',
    'attacks': [
        {
            'type': 'noise_injection',
            'intensity': 0.1,
            'affected_clients': 0.3
        }
    ]
}

print("To run a real security experiment, use:")
print("")
print("from experiment_runners.run_with_attacks import main")
print("")
print("# Configure your experiment")
print("config = {")
for key, value in example_config.items():
    print(f"    '{key}': {repr(value)},")
print("}")
print("")
print("# Run the experiment")
print("# results = main(config)")
print("")
print("⚠️  Note: Actual experiments may take significant time to complete.")

## Summary

This notebook demonstrated:

✅ **Attack Types**: Noise injection, label flipping, gradient flipping, client failures, data asymmetry, and missed classes

✅ **Security Testing**: How to configure and run experiments with different attack scenarios

✅ **Robustness Evaluation**: Comparing model performance under attack vs baseline

✅ **Strategy Comparison**: Testing which aggregation strategies are most robust

✅ **Best Practices**: Security recommendations for production federated learning systems

### Next Steps:
- Run comprehensive security evaluations on your specific use case
- Implement monitoring and detection systems
- Consider advanced defense mechanisms like differential privacy
- Test your system against adaptive and sophisticated attacks